In [1]:
import json
import urllib3
import config
import pandas as pd
import string
import datetime

http = urllib3.PoolManager()

# http://api.openweathermap.org/geo/1.0/direct?q={city name},{state code},{country code}&limit={limit}&appid={API key}
location = [
    'New York City, NY, USA',
    'Los Angeles, CA, USA',
    'Chicago, IL, USA',
    'Houston, TX, USA',
    'Phoenix, AZ, USA',
    'Philadelphia, PA, USA',
    'San Antonio, TX, USA',
    'San Diego, CA, USA',
    'Dallas, TX, USA',
    'San Jose, CA, USA',
    'Austin, TX, USA',
    'Jacksonville, FL, USA',
    'Fort Worth, TX, USA',
    'Columbus, OH, USA',
    'San Francisco, CA, USA',
    'Charlotte, NC, USA',
    'Indianapolis, IN, USA',
    'Seattle, WA, USA',
    'Denver, CO, USA',
    'Washington, D.C., DC, USA',    
]

# append to empty df doesn't happen in place; need to store it first
# https://stackoverflow.com/questions/16597265/appending-to-an-empty-dataframe-in-pandas
loc_responses = []
all_weather_data = []
weather_data = []

for l in location:
    # Location call
    call_string = 'https://api.openweathermap.org/geo/1.0/direct?q=' + l + '&limit=1&appid=' + config.api_key
    loc_response = json.loads(http.request('GET', call_string).data)
    loc_responses.append(loc_response)
    #locations_df = locations_df.append(pd.json_normalize(loc_response))
    print(l, loc_response[0]['lat'], loc_response[0]['lon'])
    # Weather call
    call_string = 'https://api.openweathermap.org/data/2.5/weather?lat=' + str(loc_response[0]['lat']) + '&lon=' + str(loc_response[0]['lon']) + '&appid=' + config.api_key
    weather_response = json.loads(http.request('GET', call_string).data)
    all_weather_data.append(weather_response)
    # useful for processing json: https://stackoverflow.com/questions/67745643/select-specific-keys-inside-a-json-using-python
    weather_trim = { 
        'city' : weather_response['name'],
        'description' : weather_response['weather'][0]['description'],
        'icon': weather_response['weather'][0]['icon'],   
        'current_temp': weather_response['main']['temp'] - 273.15,
        'feels_like': weather_response['main']['feels_like'] - 273.15,
        'temp_min': weather_response['main']['temp_min'] - 273.15,
        'temp_max': weather_response['main']['temp_max'] - 273.15,
        'visibility': weather_response['visibility'],
        'sunrise': datetime.datetime.fromtimestamp(weather_response['sys']['sunrise']),
        'sunset': datetime.datetime.fromtimestamp(weather_response['sys']['sunset']),
        'current_time':  datetime.datetime.fromtimestamp(weather_response['dt']),
        'timezone': weather_response['timezone']
    }
    weather_data.append(weather_trim)

New York City, NY, USA 40.7127281 -74.0060152
Los Angeles, CA, USA 34.0536909 -118.242766
Chicago, IL, USA 41.8755616 -87.6244212
Houston, TX, USA 29.7589382 -95.3676974
Phoenix, AZ, USA 33.4484367 -112.074141
Philadelphia, PA, USA 39.9527237 -75.1635262
San Antonio, TX, USA 29.4246002 -98.4951405
San Diego, CA, USA 32.7174202 -117.1627728
Dallas, TX, USA 32.7762719 -96.7968559
San Jose, CA, USA 37.3361663 -121.890591
Austin, TX, USA 30.2711286 -97.7436995
Jacksonville, FL, USA 30.3321838 -81.655651
Fort Worth, TX, USA 32.753177 -97.3327459
Columbus, OH, USA 39.9622601 -83.0007065
San Francisco, CA, USA 37.7790262 -122.419906
Charlotte, NC, USA 35.2272086 -80.8430827
Indianapolis, IN, USA 39.7683331 -86.1583502
Seattle, WA, USA 47.6038321 -122.330062
Denver, CO, USA 39.7392364 -104.984862
Washington, D.C., DC, USA 38.895 -77.0366


In [2]:
weather_df = pd.json_normalize(weather_data)
weather_df.head()

,city,description,icon,current_temp,feels_like,temp_min,temp_max,visibility,sunrise,sunset,current_time,timezone
0,New York,clear sky,01n,23.85,23.73,21.87,25.31,10000,2024-07-02 02:29:13,2024-07-02 17:30:48,2024-07-02 17:46:01,-14400
1,Los Angeles,clear sky,01d,28.50,29.14,22.23,32.97,10000,2024-07-02 05:45:48,2024-07-02 20:08:09,2024-07-02 17:43:51,-25200
2,Chicago,clear sky,01d,27.44,27.57,24.54,28.10,10000,2024-07-02 03:19:48,2024-07-02 18:29:10,2024-07-02 17:48:36,-18000
3,Houston,clear sky,01d,33.54,40.54,31.59,34.65,10000,2024-07-02 04:25:02,2024-07-02 18:25:54,2024-07-02 17:53:56,-18000
4,Phoenix,scattered clouds,03d,43.58,42.95,42.26,44.59,10000,2024-07-02 05:22:42,2024-07-02 19:41:53,2024-07-02 17:52:20,-25200


In [3]:
# Weather forecast
# api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API key}
all_forecasts = []
forecast_data = []

for l in loc_responses:
    call_string = 'https://api.openweathermap.org/data/2.5/forecast?lat=' + str(l[0]['lat']) + '&lon=' + str(l[0]['lon']) + '&appid=' + config.api_key
    forecast_response = json.loads(http.request('GET', call_string).data)
    all_forecasts.append(forecast_response)
    for l in forecast_response['list']:
        forecast_trim = { 
            'city' : forecast_response['city']['name'],
            'description' : l['weather'][0]['description'],
            'icon': l['weather'][0]['icon'],   
            'current_temp': l['main']['temp'] - 273.15,
            'feels_like': l['main']['feels_like'] - 273.15,
            'temp_min': l['main']['temp_min'] - 273.15,
            'temp_max': l['main']['temp_max'] - 273.15,
            'visibility': l['visibility'],
            # 'sunrise': datetime.datetime.fromtimestamp(forecast_response['city']['sunrise']),
            # 'sunset': datetime.datetime.fromtimestamp(forecast_response['city']['sunset']),
            'forecast_time':  datetime.datetime.fromtimestamp(l['dt']),
            'timezone': forecast_response['city']['timezone']
        }
        forecast_data.append(forecast_trim)

In [4]:
forecast_df = pd.json_normalize(forecast_data)
forecast_df.head()

,city,description,icon,current_temp,feels_like,temp_min,temp_max,visibility,forecast_time,timezone
0,New York,scattered clouds,03n,23.57,23.39,23.02,23.57,10000,2024-07-02 20:00:00,-14400
1,New York,broken clouds,04n,22.38,22.16,21.64,22.38,10000,2024-07-02 23:00:00,-14400
2,New York,broken clouds,04n,20.46,20.13,20.46,20.46,10000,2024-07-03 02:00:00,-14400
3,New York,scattered clouds,03d,22.17,21.85,22.17,22.17,10000,2024-07-03 05:00:00,-14400
4,New York,overcast clouds,04d,25.61,25.27,25.61,25.61,10000,2024-07-03 08:00:00,-14400


In [5]:
final_df = forecast_df.merge(weather_df, on = 'city', how = 'inner',  suffixes=('_forecast', '_current'))
final_df.to_excel('current_forecast_weather.xlsx')